In [16]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [17]:
!pip install dotenv

In [18]:
from google.colab import files

uploaded = files.upload()

Saving .env to .env (2)


In [19]:
from dotenv import load_dotenv
import os
load_dotenv(".env")

True

In [20]:
gemini_api_key = os.getenv("GEMINI_API_KEY")
sender_email = os.getenv("EMAIL")
sender_password = os.getenv("EMAIL_PASSWORD")


In [21]:
!pip install langchain_google_genai
!pip install langchain_community
!pip install langchain langgraph openpyxl google-generativeai
!pip install yagmail

  Using cached google_ai_generativelanguage-0.6.18-py3-none-any.whl.metadata (9.8 kB)
Using cached google_ai_generativelanguage-0.6.18-py3-none-any.whl (1.4 MB)
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.15
    Uninstalling google-ai-generativelanguage-0.6.15:
      Successfully uninstalled google-ai-generativelanguage-0.6.15
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.18 which is incompatible.


  Using cached google_ai_generativelanguage-0.6.15-py3-none-any.whl.metadata (5.7 kB)
Using cached google_ai_generativelanguage-0.6.15-py3-none-any.whl (1.3 MB)
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.18
    Uninstalling google-ai-generativelanguage-0.6.18:
      Successfully uninstalled google-ai-generativelanguage-0.6.18
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-google-genai 2.1.5 requires google-ai-generativelanguage<0.7.0,>=0.6.18, but you have google-ai-generativelanguage 0.6.15 which is incompatible.


In [22]:
from langchain.prompts import (
    PromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    ChatPromptTemplate,
)
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings

In [23]:
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash-001",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    google_api_key=gemini_api_key
)

embeddings = GoogleGenerativeAIEmbeddings(
    model="models/embedding-001",
    google_api_key = gemini_api_key
)

In [24]:
from openpyxl import load_workbook

def load_emails(file_path, email_column_name="email"):
    wb = load_workbook(filename=file_path)
    sheet = wb.active

    header = [cell.value for cell in sheet[1]]
    if email_column_name not in header:
        raise ValueError(f"'{email_column_name}' column not found in Excel sheet.")

    col_idx = header.index(email_column_name) + 1

    emails = []
    for row in sheet.iter_rows(min_row=2, min_col=col_idx, max_col=col_idx):
        cell_value = row[0].value
        if cell_value:
            emails.append(cell_value.strip())

    return emails


In [30]:
load_emails("/content/drive/MyDrive/genai/langgraph/email agent/emails.xlsx")

['vibhor.1bhatia@gmail.com']

In [49]:
import yagmail

def send_email(to, subject, body, sender_email, sender_password):
  yag = yagmail.SMTP(user=sender_email, password=sender_password)
  yag.send(to=to, subject=subject, contents=body)
  yag.close()


In [26]:
email_content = "test emal content"

In [50]:
from langgraph.graph import StateGraph, END
from typing import TypedDict

class EmailAgentState(TypedDict):
  emails: list
  content: str

def read_emails_node(state):
  emails = load_emails("/content/drive/MyDrive/genai/langgraph/email agent/emails.xlsx")
  print(emails)
  return {**state, "emails": emails}

def generate_content_node(state):
  print(state)
  content = email_content
  return {**state, "content": content}

def send_emails_node(state):
  print(state)
  emails = state.get("emails", [])
  content = state.get("content", "")

  for email in emails:
      send_email(email, "Automated Email", content, sender_email, sender_password)
  return state


graph = StateGraph(EmailAgentState)
graph.add_node("read_emails", read_emails_node)
graph.add_node("generate_content", generate_content_node)
graph.add_node("send_emails", send_emails_node)

graph.set_entry_point("read_emails")
graph.add_edge("read_emails", "generate_content")
graph.add_edge("generate_content", "send_emails")
graph.set_finish_point("send_emails")

compiled_graph = graph.compile()
final_state = compiled_graph.invoke({})


['vibhor.1bhatia@gmail.com']
{'emails': ['vibhor.1bhatia@gmail.com']}
{'emails': ['vibhor.1bhatia@gmail.com'], 'content': 'Hi, this is a test email sent via my LangGraph agent.'}


FileNotFoundError: [Errno 2] No such file or directory: '/root/.yagmail'